# pyLDAvis

[`pyLDAvis`](https://github.com/bmabey/pyLDAvis) is a python libarary for interactive topic model visualization.
It is a port of the fabulous [R package](https://github.com/cpsievert/LDAvis>) by Carson Sievert and Kenny Shirley.  They did the hard work of crafting an effective visualization. `pyLDAvis` makes it easy to use the visualiziation from Python and, in particualr, IPython notebooks. To learn more about the method behind the visualization I suggest reading the [original paper](http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf) explaining it.

This notebook provides a quick overview of how to use `pyLDAvis`. Refer to the [documenation](https://pyldavis.readthedocs.org/en/latest/) for details.


## BYOM - Bring your own model

`pyLDAvis` is agnostic to how your model was trained. To visualize it you need to provide the topic-term distribtuions, document-topic distributions, and basic information about the corpus which the model was trained on. The main function is the [`prepare`](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.prepare) function that will transform your data into the format needed for the visualization.

Below we load a model trained in R and then visualize it. The model was trained on a corpus of 2000 movie reviews parsed by [Pang and Lee (ACL, 2004)](http://www.cs.cornell.edu/people/pabo/movie-review-data/), originally gathered from the IMDB archive of the rec.arts.movies.reviews newsgroup.

In [8]:
import json
import numpy as np

def load_R_model(filename):
    with open(filename, 'r') as j:
        data_input = json.load(j)
    data = {'topic_term_dists': data_input['phi'], 
            'doc_topic_dists': data_input['theta'],
            'doc_lengths': data_input['doc.length'],
            'vocab': data_input['vocab'],
            'term_frequency': data_input['term.frequency']}
    return data

movies_model_data = load_R_model('data/movie_reviews_input.json')

print('Topic-Term shape: %s' % str(np.array(movies_model_data['topic_term_dists']).shape))
print('Doc-Topic shape: %s' % str(np.array(movies_model_data['doc_topic_dists']).shape))

Topic-Term shape: (20, 14567)
Doc-Topic shape: (2000, 20)


Now that we have the data loaded we use the `prepare` function:

In [9]:
import pyLDAvis
movies_vis_data = pyLDAvis.prepare(**movies_model_data)

Once you have the visualization data prepared you can do a number of things with it. You can [save the vis](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.save_html) to an stand-alone HTML file, [serve it](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.show), or [dispaly it](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.display) in the notebook. Let's go ahead and display it:

In [10]:
pyLDAvis.display(movies_vis_data)

Pretty, huh?! Again, you should be thanking the original [LDAvis people](https://github.com/cpsievert/LDAvis) for that. You may thank me for the IPython integartion though. :)

To see other models visualzied check out [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Movie%20Reviews,%20AP%20News,%20and%20Jeopardy.ipynb).

*ProTip:* To avoid tediously typing in `display` all the time use:

In [11]:
pyLDAvis.enable_notebook()

## Making the common case easy - Gensim and others!

Built on top of the generic `prepare` function are helper functions for [gensim](https://radimrehurek.com/gensim/) and [GraphLab Create](https://dato.com/products/create/). To demonstrate below I am loading up a trained gensim model and coresponding dictionary and corpus (see [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Gensim%20Newsgroup.ipynb) for how these were created):

In [12]:
import gensim

dictionary = gensim.corpora.Dictionary.load('newsgroups.dict')
corpus = gensim.corpora.MmCorpus('newsgroups.mm')
lda = gensim.models.ldamodel.LdaModel.load('newsgroups_50.model')

In the dark ages in order to inspect our topics all we had was `show_topics` and friends:

In [13]:
lda.show_topics()

[u'0.017*minnesota + 0.014*income + 0.012*morris + 0.012*roy + 0.012*partners + 0.010*francis + 0.008*amounts + 0.007*gear + 0.007*antenna + 0.007*motorola',
 u"0.015*don't + 0.014*know + 0.012*said + 0.011*time + 0.011*one + 0.010*didn't + 0.009*get + 0.008*like + 0.008*i'm + 0.007*going",
 u'0.016*university + 0.016*article + 0.015*pat + 0.012*john + 0.012*tony + 0.010*greg + 0.010*orbit + 0.009*toronto + 0.009*gerald + 0.009*steve',
 u'0.018*engine + 0.017*moon + 0.014*senate + 0.012*front + 0.011*car + 0.011*new + 0.010*bmw + 0.009*dod + 0.009*ford + 0.008*honda',
 u'0.022*president + 0.018*government + 0.015*clinton + 0.013*white + 0.011*house + 0.011*security + 0.010*secret + 0.010*clipper + 0.009*david + 0.009*encryption',
 u'0.026*bit + 0.023*scsi + 0.021*chip + 0.018*mac + 0.018*speed + 0.015*data + 0.011*chips + 0.011*mhz + 0.010*fast + 0.009*serial',
 u'0.025*greece + 0.024*doug + 0.023*energy + 0.022*blood + 0.014*jew + 0.013*judas + 0.012*ron + 0.011*murray + 0.010*hiv + 0

Thankfully, in addition to these *still helpful functions*, we can get a feel for all of the topics with this one-liner:

In [14]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     10.007999        1       1 -0.208404 -0.086964
30      7.947768        1       2 -0.167611 -0.084675
49      5.548640        1       3 -0.078181  0.243878
33      4.529078        1       4  0.276197 -0.021424
23      4.402293        1       5 -0.185674 -0.121984
35      3.721980        1       6 -0.151227  0.099506
39      3.535019        1       7 -0.139642 -0.095885
12      3.153871        1       8 -0.048989  0.131109
34      3.150678        1       9 -0.005706 -0.159566
43      2.782393        1      10 -0.072772 -0.033457
...          ...      ...     ...       ...       ...
47      0.740943        1      41  0.259656 -0.059987
6       0.717105        1      42 -0.006820 -0.016676
45      0.630999        1      43  0.091218 -0.019896
31      0.610080        1      44  0.082820 -0.029565
37      0.587591        1      45  0.056591 -0.040092
9       0.569504        1      46  0.207493  0.002920
18      0.566455        1      47  0.190914 -0.045167
1       0.522890        1      48  0.130360  0.093153
13      0.489081        1      49  0.154583  0.013493
15      0.093762        1      50  0.248105 -0.017513

[50 rows x 5 columns], topic_info=      Category          Freq     Term  Total  loglift  logprob
8306   Default  62049.000000      'ax  62049  30.0000  30.0000
9344   Default   4771.000000      max   4771  29.0000  29.0000
10443  Default   5498.000000  posting   5498  28.0000  28.0000
15347  Default   4991.000000     host   4991  27.0000  27.0000
12346  Default   4809.000000     nntp   4809  26.0000  26.0000
14033  Default   2621.000000      god   2621  25.0000  25.0000
19717  Default   3370.000000      edu   3370  24.0000  24.0000
9710   Default   1666.000000      key   1666  23.0000  23.0000
2100   Default   7436.000000  article   7436  22.0000  22.0000
6223   Default   5850.000000   people   5850  21.0000  21.0000
...        ...           ...      ...    ...      ...      ...
10026  Topic50     12.716612      duh     18   6.5368  -4.8637
11367  Topic50      2.781656      'it      6   6.3820  -6.1171
14380  Topic50      3.111066      pmc      7   6.2411  -6.1038
8295   Topic50      3.490618      nrk      9   6.0904  -6.0032
17549  Topic50      3.609741      pry     10   6.0087  -5.9796
7162   Topic50      3.058289      gwm      8   5.9624  -6.2490
9344   Topic50    231.361908      max   4771   3.9261  -1.8944
19983  Topic50      7.160039      tad     46   5.0583  -5.4039
12502  Topic50      3.239009      pax     13   5.7479  -5.9780
1871   Topic50      4.072450      sax     15   5.5576  -6.0251

[3687 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
11498     21  0.857143         #are
19651     21  0.800000          #as
4995      43  1.000000    #compared
16088      4  0.960000      #email#
4503       8  0.356846     #email's
4503      26  0.008299     #email's
4503      32  0.580913     #email's
4503      39  0.049793     #email's
17788      2  0.888889        #from
16037     43  1.000000  #homosexual
...      ...       ...          ...
3818      41  0.022088        young
3818      44  0.032129        young
9668      11  1.000000   yugoslavia
7315      32  0.857143       zaurak
1743      25  0.909091   zeineldine
3048      36  0.967742         zeus
16679      1  0.979592      zionist
6387      40  0.994186         zone
20961     15  0.024096      zoology
20961     25  0.963855      zoology

[13246 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 31, 50, 34, 24, 36, 40, 13, 35, 44, 27, 6, 49, 42, 5, 29, 11, 37, 15, 39, 28, 30, 4, 12, 23, 45, 25, 47, 43, 22, 9, 33, 17, 20, 8, 41, 26, 21, 1, 3, 48, 7, 46, 32, 38, 10, 19, 2, 14, 16])

## GraphLab

As I mentioned above you can also easily visualize GraphLab TopicModels as well. Check out [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/GraphLab.ipynb#topic=7&lambda=0.41&term=) if you are interested in that.


## Go forth and visualize!

What are you waiting for? Go ahead and `pip install pyldavis`.